# This script plots the SEA results of mean sigma. 
# Sigma less than 0.5 is transparent.

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.lines import Line2D

# Configuration parameters
slt_range = (18, 23)
mag_lat_min = -10
mag_lat_max = 10
lon_min = 0
lon_max = 70
granularity_hours = 2
sigma_index_threshold = 0.5  # Threshold for Sigma Index to identify plasma bubbles
save_directory = "/Users/gildagonzalez/Documents/Datos_Plots/Sigma_Index/SEA_Sigma/Test"
os.makedirs(save_directory, exist_ok=True)
print(f"Directory '{save_directory}' is ready for saving plots.")

# Data file paths
combined_data_file_path = '/Users/gildagonzalez/Documents/Datos_Plots/Storms/Datos geomag/Geo_data.txt'
sigma_index_data_file_path = '/Users/gildagonzalez/Documents/Datos_Plots/Sigma_Index/SigmaIndexOrbits/SigmaIndexOrbits.dat'

def load_combined_data(file_path):
    """Load and preprocess combined geomagnetic data."""
    try:
        df = pd.read_csv(file_path, sep="\t", dtype=str, skiprows=1,
                         names=['YEAR', 'D', 'h', 'm', 'Bz', 'Ey', 'SYM-H', 'SME'])
        if df.iloc[0]['YEAR'].startswith('YEAR'):
            df = df.iloc[1:]
        df.replace({'9999.99': np.nan, '999.99': np.nan}, inplace=True)
        df[['Bz', 'Ey', 'SYM-H', 'SME']] = df[['Bz', 'Ey', 'SYM-H', 'SME']].apply(pd.to_numeric, errors='coerce')
        df['datetime'] = pd.to_datetime(df['YEAR'].str.zfill(4) + df['D'].str.zfill(3) + df['h'].str.zfill(2) + df['m'].str.zfill(2), format='%Y%j%H%M')
        df.set_index('datetime', inplace=True)
        df.drop(['YEAR', 'D', 'h', 'm'], axis=1, inplace=True)
        df = df.resample('h').mean()
        return df
    except Exception as e:
        print(f"Error loading combined data: {e}")
        return None

def identify_storm_events(sym_h_series, threshold=-50):
    """Identify storm events based on the SYM-H index threshold."""
    return sym_h_series[sym_h_series <= threshold]

def find_min_sym_h_within_events(sym_h_data, storm_events, window_days=1):
    """Find the minimum SYM-H value within a specified window around each storm event."""
    min_sym_h_times = []
    for event_time in storm_events.index:
        window_data = sym_h_data[event_time - pd.Timedelta(days=window_days):event_time + pd.Timedelta(days=window_days)]
        if not window_data.empty:
            min_sym_h_times.append(window_data.idxmin())
    return min_sym_h_times

def calculate_sea_results(data, param, min_sym_h_times, window_days):
    """Calculate superposed epoch analysis (SEA) results."""
    sea_results = pd.DataFrame()
    for min_time in min_sym_h_times:
        window_data = data[min_time - pd.Timedelta(days=window_days):min_time + pd.Timedelta(days=window_days)].copy()
        if param in window_data.columns:
            window_data[param] = pd.to_numeric(window_data[param], errors='coerce')
            window_data['Days Since Min SYM-H'] = (window_data.index - min_time) / pd.Timedelta(days=1)
            sea_results = pd.concat([sea_results, window_data[['Days Since Min SYM-H', param]]])
    if not sea_results.empty:
        sea_results = sea_results.groupby('Days Since Min SYM-H')[param].agg(['mean', 'std']).reset_index()
    return sea_results

def plot_sea_results(sea_results, param, y_label, save_directory):
    """Plot superposed epoch analysis (SEA) results."""
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(sea_results['Days Since Min SYM-H'], sea_results['mean'], label='Mean', color='black', linewidth=2)
    ax.fill_between(sea_results['Days Since Min SYM-H'], sea_results['mean'] - sea_results['std'], sea_results['mean'] + sea_results['std'], color='lightgray', alpha=0.5, label='Std Dev')
    ax.axvline(x=0, color='red', linestyle='--', label='Zero Epoch')
    ax.legend(loc='upper right')
    ax.set_xlabel('Days', fontsize=18)
    ax.set_ylabel(y_label, fontsize=18)
    ax.tick_params(axis='x', labelsize=18)
    ax.tick_params(axis='y', labelsize=18)
    ax.grid(True)
    plt.tight_layout()
    plot_filename = f"{save_directory}/SEA_Result_{param}.png"
    plt.savefig(plot_filename)
    plt.close(fig)

def load_sigma_index_data(file_path, mag_lat_min, mag_lat_max, lon_min, lon_max, slt_range):
    """Load and filter Sigma Index data based on specified criteria."""
    try:
        df = pd.read_csv(file_path, sep='\t', dtype=str, skiprows=1,
                         names=['Year', 'DDD', 'HH', 'MM', 'Sigma_Index', 'Solar_Local_Time', 'Longitude', 'Magnetic_Latitude', 'ICON_L27_Orbit_Number'])
        df[['Sigma_Index', 'Solar_Local_Time', 'Longitude', 'Magnetic_Latitude', 'ICON_L27_Orbit_Number']] = df[['Sigma_Index', 'Solar_Local_Time', 'Longitude', 'Magnetic_Latitude', 'ICON_L27_Orbit_Number']].apply(pd.to_numeric, errors='coerce')
        df['datetime'] = pd.to_datetime(df[['Year', 'DDD', 'HH', 'MM']].astype(str).agg(' '.join, axis=1), format='%Y %j %H %M')
        df.set_index('datetime', inplace=True)
        df.drop(['Year', 'DDD', 'HH', 'MM'], axis=1, inplace=True)
        df_filtered = df[(df['Magnetic_Latitude'] >= mag_lat_min) & (df['Magnetic_Latitude'] <= mag_lat_max) &
                         (df['Longitude'] >= lon_min) & (df['Longitude'] <= lon_max) &
                         (df['Solar_Local_Time'] >= slt_range[0]) & (df['Solar_Local_Time'] <= slt_range[1])]
        return df_filtered
    except Exception as e:
        print(f"Error loading and filtering sigma index data: {e}")
        return None

def plot_sigma_index_over_time(sigma_index_data, min_sym_h_times, save_directory, window_days=3, marker_size=50):
    """Plot Sigma Index data over time using superposed epoch analysis."""
    time_unit = 'Days Since Min SYM-H'
    sea_results = pd.DataFrame()

    for min_time in min_sym_h_times:
        window_data = sigma_index_data[min_time - pd.Timedelta(days=window_days):min_time + pd.Timedelta(days=window_days)].copy()
        window_data[time_unit] = (window_data.index - min_time) / pd.Timedelta(days=1)
        mean_sigma = window_data.groupby(['Solar_Local_Time', time_unit])['Sigma_Index'].mean().reset_index(name='mean_sigma')
        sea_results = pd.concat([sea_results, mean_sigma])

    sea_results = sea_results.groupby(['Solar_Local_Time', time_unit])['mean_sigma'].mean().reset_index()

    fig, ax = plt.subplots(figsize=(10, 5))
    for _, row in sea_results.iterrows():
        color = 'gray' if row['mean_sigma'] < 0.5 else 'red'
        alpha = 0.2 if row['mean_sigma'] < 0.5 else 1.0
        ax.scatter(row[time_unit], row['Solar_Local_Time'], color=color, s=marker_size, alpha=alpha)

    legend_elements = [
        Line2D([0], [0], marker='.', color='w', markerfacecolor='gray', markersize=10, label='Sigma < 0.5', alpha=0.2),
        Line2D([0], [0], marker='.', color='w', markerfacecolor='red', markersize=10, label='Sigma >= 0.5')
    ]
    ax.legend(handles=legend_elements, loc='upper right')

    mag_lat_text = f'Magnetic Latitude: {mag_lat_min} to {mag_lat_max}'
    plt.annotate(mag_lat_text, xy=(0.05, 0.95), xycoords='axes fraction', fontsize=12, backgroundcolor='white')

    ax.axvline(x=0, color='black', linestyle='--', label='Zero Epoch')
    ax.set_xlabel('Days Since Min SYM-H', fontsize=18)
    ax.set_ylabel('Solar Local Time', fontsize=18)
    ax.set_xlim([-3, 3])
    ax.set_ylim(slt_range)
    ax.tick_params(axis='x', labelsize=18)
    ax.tick_params(axis='y', labelsize=18)
    ax.grid(True)
    plt.tight_layout()
    plot_filename = f"{save_directory}/Sigma_Index_Superposed_Epoch_Africa_4.png"
    plt.savefig(plot_filename)
    plt.close(fig)

def run_analysis(data_file, sigma_index_data_file, save_directory, slt_range, lon_min, lon_max, marker_size=50):
    """Run the complete analysis."""
    combined_data = load_combined_data(data_file)
    sigma_index_data = load_sigma_index_data(sigma_index_data_file, mag_lat_min, mag_lat_max, lon_min, lon_max, slt_range)
    
    if combined_data is not None and sigma_index_data is not None:
        sym_h_hourly = combined_data['SYM-H'].resample('h').mean()
        storm_events = identify_storm_events(sym_h_hourly, -50)
        min_sym_h_times = find_min_sym_h_within_events(sym_h_hourly, storm_events)
        
        # Perform and plot SEA results for geomagnetic parameters
        y_labels = {'SYM-H': 'Mean SYM-H (nT)', 'Bz': 'Mean Bz (nT)', 'Ey': 'Mean Ey (mV/m)', 'SME': 'Mean SME (nT)'}
        for param, y_label in y_labels.items():
            sea_results = calculate_sea_results(combined_data, param, min_sym_h_times, 3)
            if not sea_results.empty:
                plot_sea_results(sea_results, param, y_label, save_directory)

        # Plot Sigma Index data
        plot_sigma_index_over_time(sigma_index_data, min_sym_h_times, save_directory, 3, marker_size)

# Run the analysis
run_analysis(combined_data_file_path, sigma_index_data_file_path, save_directory, slt_range, lon_min, lon_max, marker_size=50)


Directory '/Users/gildagonzalez/Documents/Datos_Plots/Sigma_Index/SEA_Sigma/Test' is ready for saving plots.
